In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import os.path
import sys, getopt

from time import time
import torch
from torch import nn
import torch.nn.functional as F
import math

from torch import Tensor
from torchvision.transforms import Compose, Resize, ToTensor

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
import argparse
from ruamel.yaml import YAML
import pathlib
sys.path.append('realistic_fl_noise')
sys.path.append('utils')
from loss_functions import *
from naf_net_loading import replace_layers

import random
#sys.path.append('realistic_fl_noise')
#from realistic_fl_dataloader import OL_Full_Noise,collate_fn
sys.path.append('..')

from make_dataset import make_dataset
from make_models import make_model

import argparse
from ruamel.yaml import YAML
import pathlib

import matplotlib.pyplot as plt

device = "cuda:0"


In [2]:
df_full = pd.read_pickle("results/metric_results.pkl")

In [3]:

ss = df_full.index.get_level_values('S').unique()
ss=np.sort(ss.to_list())
bs = df_full.index.get_level_values('B').unique()
bs=np.sort(bs.to_list())

lf=df_full.iloc[0]["losses"]
ranges_to_remove = [[155,165],[293,302]]
indexes_all = [i for i in range(len(lf.errors)) if not np.any([ k<=i and i<=l for k,l in ranges_to_remove])]
indexes_23 = [i for i in range(155)]
indexes_24 = [i for i in range(156,len(lf.errors)) if not np.any([ k<=i and i<=l for k,l in ranges_to_remove])]

#indexes_short = [i for i in range(0,77)]

indexes = indexes_all


In [4]:
%matplotlib notebook
val_dict = {}

lin_v_dict = {}
metric = "LPIPS"
metric_ind = [i for i,name in enumerate(lf.error_names) if name == metric][0]

for model in df_full.index.get_level_values('model').unique():
    vals = np.zeros((len(bs),len(ss)))
    lin_v=[]
    lin_s=[]
    lin_b = []
    lin_bt=[]
    for i, s in enumerate(ss):
        for j, b in enumerate(bs):
            lf = df_full.loc[model,s,b]['losses']
            vals[j,i] = lf.errors[indexes,metric_ind].mean()/100
            lin_v.append(lf.errors[indexes,metric_ind].mean()/100)
            lin_s.append(s)
            lin_b.append(b)
            lin_bt.append(b*s)
    val_dict[model]=vals
    lin_v_dict[model]=lin_v

In [ ]:
from scipy import interpolate
new_vs_list=[]
models = df_full.index.get_level_values('model').unique()
for model in models:
    lin_v = lin_v_dict[model]
    new_ss = ss
    new_bs = bs
    new_v = val_dict[model].transpose()

    plt.figure()
    plt.imshow(new_v,extent=[min(new_bs),max(new_bs),max(new_ss),min(new_ss)],aspect="auto")
    plt.xlabel("LLL Ratio")
    plt.ylabel("Signal")
    plt.colorbar()
    plt.title(model)
    plt.show()
    
    new_vs_list.append(new_v)

In [ ]:
#import seaborn as sns
#sns.set_theme()
new_vs_list=np.array(new_vs_list)
plt.figure()
plt.imshow(np.min(new_vs_list,axis=0),extent=[min(new_bs),max(new_bs),max(new_ss),min(new_ss)],aspect="auto")
plt.xlabel("LLL Ratio")
plt.ylabel("Signal")
plt.title("Best Performing Model (LPIPS)")
save_path = "/z/Proj/compoptics/Trevors-Data/2021-07-20/transformer_models/chtc/figure_results/figures/quant/"
#plt.savefig(save_path+"LPIPS_best_model_6x5.svg")
plt.show()


In [ ]:
%matplotlib notebook
val_dict = {}

lin_v_dict = {}
metric = "LPIPS"
metric_ind = [i for i,name in enumerate(lf.error_names) if name == metric][0]
models = df_full.index.get_level_values('model').unique()
for i, s in enumerate(ss):
    for j, b in enumerate(bs):
        plt.figure()
        lf2=df_full.loc[models[0],s,b]['losses'].errors[indexes,metric_ind]/100

        for model in models:
            lf = df_full.loc[model,s,b]['losses']
            plt.plot(indexes,lf.errors[indexes,metric_ind]/100)#-lf2)

        plt.title(f"S:{s}, B:{b}")
        plt.legend(models)
        plt.show()